<a href="https://colab.research.google.com/github/naturalsolutions/3m-ml-data-preparation/blob/main/3m_landcover_ai_data_preparation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Depedencies


In [48]:
!pip install rasterio
!pip install geocube
!pip install pandas
!pip install geopandas
!pip install shapely
!pip install h5
!pip install numpy


DEPRECATION: geopolars 0.1.0a4 has a non-standard dependency specifier pyarrow>=4.0.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of geopolars or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
DEPRECATION: geopolars 0.1.0a4 has a non-standard dependency specifier pyarrow>=4.0.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of geopolars or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
DEPRECAT

In [ ]:
# Running in Colab ? Mounting drive and set working directory
from google.colab import drive

drive.mount("/content/drive")
WORKDIR = "/content/drive/MyDrive/3m-ai/grid_256px_11_classes"

In [49]:
# As an alternative, work locally
WORKDIR = "./3m-ai/grid_256px_11_classes"

# Utils functions:


### - Grid factories


In [50]:
from typing import List

# from pandera.typing import Index, DataFrame, Series
# from pandera.typing.geopandas import GeoDataFrame, GeoSeries
# import pandera as pa


def create_grid(
    gdf: None,
    bounds: List[float] | None = None,
    n_cells=10,
    overlap=False,
    crs="EPSG:4326",
):
    """Create square grid that covers a geodataframe area
    or a fixed boundary with x-y coords
    returns: a GeoDataFrame of grid polygons
    see https://james-brennan.github.io/posts/fast_gridding_geopandas/
    """

    import geopandas as gpd
    import shapely
    import numpy as np

    if bounds != None:
        xmin, ymin, xmax, ymax = bounds
    else:
        xmin, ymin, xmax, ymax = gdf.total_bounds

    # get cell size
    cell_size = (xmax - xmin) / n_cells
    # create the cells in a loop
    grid_cells = []
    for x0 in np.arange(xmin, xmax + cell_size, cell_size):
        for y0 in np.arange(ymin, ymax + cell_size, cell_size):
            x1 = x0 - cell_size
            y1 = y0 + cell_size
            poly = shapely.geometry.box(x0, y0, x1, y1)
            # print (gdf.overlay(poly, how='intersection'))
            grid_cells.append(poly)

    cells = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs=crs)
    if overlap == True:
        cols = ["grid_id", "geometry", "grid_area"]
        cells = cells.sjoin(gdf, how="inner").drop_duplicates("geometry")
    return cells


def create_hex_grid(gdf=None, bounds=None, n_cells=10, overlap=False, crs="EPSG:4326"):
    """Hexagonal grid over geometry.
    See https://sabrinadchan.github.io/data-blog/building-a-hexagonal-cartogram.html
    """

    from shapely.geometry import Polygon
    import geopandas as gpd
    import numpy as np

    if bounds != None:
        xmin, ymin, xmax, ymax = bounds
    else:
        xmin, ymin, xmax, ymax = gdf.total_bounds

    unit = (xmax - xmin) / n_cells
    a = np.sin(np.pi / 3)
    cols = np.arange(np.floor(xmin), np.ceil(xmax), 3 * unit)
    rows = np.arange(np.floor(ymin) / a, np.ceil(ymax) / a, unit)

    # print (len(cols))
    hexagons = []
    for x in cols:
        for i, y in enumerate(rows):
            if i % 2 == 0:
                x0 = x
            else:
                x0 = x + 1.5 * unit

            hexagons.append(
                Polygon(
                    [
                        (x0, y * a),
                        (x0 + unit, y * a),
                        (x0 + (1.5 * unit), (y + unit) * a),
                        (x0 + unit, (y + (2 * unit)) * a),
                        (x0, (y + (2 * unit)) * a),
                        (x0 - (0.5 * unit), (y + unit) * a),
                    ]
                )
            )

    grid = gpd.GeoDataFrame({"geometry": hexagons}, crs=crs)
    grid["grid_area"] = grid.area
    grid = grid.reset_index().rename(columns={"index": "grid_id"})
    if overlap == True:
        cols = ["grid_id", "geometry", "grid_area"]
        grid = grid.sjoin(gdf, how="inner").drop_duplicates("geometry")
    return grid


def create_size_based_grid(
    gdf: None,
    bounds: List[float] | None = None,
    size=256,
    predicate="within",
    crs="EPSG:2154",
    epsg: int | None = None,
):
    """Create square grid of fixed size (beware of crs unit) that covers a geodataframe area
    or a fixed boundary with x-y coords
    returns: a GeoDataFrame of grid polygons
    """
    from shapely import Polygon
    from geopandas import GeoDataFrame, sjoin
    from numpy import arange

    gdf_to_merge = gdf
    gdf_to_merge["merge"] = 1
    gdf_merged = gdf_to_merge.dissolve("merge")

    if bounds != None:
        xmin, ymin, xmax, ymax = bounds
    else:
        xmin, ymin, xmax, ymax = gdf_merged.total_bounds

    cols = list(arange(xmin, xmax + size, size))
    rows = list(arange(ymin, ymax + size, size))

    polygons = []
    for x in cols[:-1]:
        for y in rows[:-1]:
            polygons.append(
                Polygon(
                    [
                        (x, y),
                        (x + size, y),
                        (x + size, y + size),
                        (x, y + size),
                    ]
                )
            )
    if epsg != None:
        grid = GeoDataFrame({"geometry": polygons}, epsg=epsg)
    else:
        grid = GeoDataFrame({"geometry": polygons}, crs=crs)
    if predicate != None:
        grid = sjoin(grid, gdf_merged, how="inner", predicate=predicate)
    grid.insert(1, "id", range(1, 1 + len(grid)))
    grid = grid.reset_index()
    grid = grid[["id", "geometry"]]
    return grid

### - Geoportail webservices


In [51]:
from typing import Union, List

Num = Union[int, float]


def stringify_bbox(bounds: List[Num]) -> str:
    """
    Receive bounding box as a list and return it as a formatted string
    """
    xmin, ymin, xmax, ymax = bounds
    return f"{xmin},{ymin},{xmax},{ymax}"


def get_tile_url(
    base_url="https://wxs.ign.fr/ortho/geoportail/r/wms",
    layer_name="ORTHOIMAGERY.ORTHOPHOTOS.BDORTHO",
    bbox="785802.3085941937752,6336806.404346071184,786208.1537742479704,6337311.468582574278",
    size=256,
) -> str:
    """
    Helper to format geoservices web expert URL

    Args:
      base_url:  "https://wxs.ign.fr/ortho/geoportail/r/wms" | "https://wxs.ign.fr/orthohisto/geoportail/r/wms"
      layer_name: layer service; see IGN geoservices doc for details:
              - Ortho https://geoservices.ign.fr/services-web-experts-ortho
              - Orthohisto https://geoservices.ign.fr/services-web-experts-orthohisto
      bbox: bounding box param, see `stringify_bbox` function to get formatted parrams from geopandas bounding box

    Returns:
      Formatted URI with params
    """
    params = f"SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&BBOX={bbox}&CRS=EPSG:2154&WIDTH={size}&HEIGHT={size}&LAYERS={layer_name}&STYLES=&FORMAT=image/geotiff&DPI=72&MAP_RESOLUTION=72&FORMAT_OPTIONS=dpi:72"
    return f"{base_url}?{params}"


def download_image_from_url(
    url=None, index=None, out_path="./downloads/tiles", default_images_folder="default"
):
    import time
    from urllib.parse import urlparse, parse_qs
    import requests
    import os

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    if url is None:
        raise ValueError("url param s required")
    t0 = time.time()
    parsed_url = urlparse(url)
    layer_param = parse_qs(parsed_url.query)["LAYERS"][0]
    bbox_param = parse_qs(parsed_url.query)["BBOX"][0]
    bbox_tuple = tuple(map(float, bbox_param.split(",")))

    if layer_param in [
        "ORTHOIMAGERY.ORTHOPHOTOS.IRC",
        "ORTHOIMAGERY.ORTHOPHOTOS.IRC-EXPRESS.2021",
    ]:
        out_dir = f"{out_path}/irc"
    elif layer_param in [
        "ORTHOIMAGERY.ORTHOPHOTOS.BDORTHO",
        "ORTHOIMAGERY.ORTHOPHOTOS.ORTHO-EXPRESS.2021",
    ]:
        out_dir = f"{out_path}/ortho"
    else:
        out_dir = f"{out_path}/{default_images_folder}"

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    try:
        r = requests.get(url)
        if r.status_code == 200:
            if index is None:
                image_path = f"{out_dir}/image_{hash(bbox_tuple)}.tiff"
            else:
                image_path = f"{out_dir}/image_{index}.tiff"
            with open(image_path, "wb") as f:
                f.write(r.content)
            return (image_path, time.time() - t0, url)
        else:
            return (None, None, url)
    except Exception as e:
        print("Exception in download_url():", e)

### - Plotting


In [52]:
def plot_trio(ortho, irc, mask):
    import rasterio
    import matplotlib.pyplot as plt

    # Paths to the three raster images
    image_paths = [ortho, irc, mask]

    # Create a figure with three subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Loop through the image paths and plot each image
    for i, image_path in enumerate(image_paths):
        with rasterio.open(image_path) as src:
            # Read the image data
            img_data = src.read()

            # Plot the image using imshow
            axes[i].imshow(
                img_data.transpose((1, 2, 0))
            )  # Transpose to (height, width, bands) for RGB images
            axes[i].set_title(f"Image {i + 1}")

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()


def plot_trio_from_index(index, dataset_path="./datasets"):
    import rasterio
    import matplotlib.pyplot as plt

    # Paths to the three raster images
    image_paths = [
        f"{dataset_path}/images/ortho/image_{index}.tiff",
        f"{dataset_path}/images/irc/image_{index}.tiff",
        f"{dataset_path}/masks/mask_{index}.tif",
    ]

    # Create a figure with three subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Loop through the image paths and plot each image
    for i, image_path in enumerate(image_paths):
        with rasterio.open(image_path) as src:
            # Read the image data
            img_data = src.read()

            # Plot the image using imshow
            axes[i].imshow(
                img_data.transpose((1, 2, 0))
            )  # Transpose to (height, width, bands) for RGB images
            axes[i].set_title(f"Image {i + 1}")

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()


def plot_area_and_grid(area_gdf, grid_gdf, title="Geoplot 1"):
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(14, 7))
    area_gdf.plot(ec="gray", fc="none", figsize=(10, 10), ax=ax)
    grid_gdf.plot(fc="none", ec="black", ax=ax)
    ax.set_title(title)

### - Data transformation utils


In [53]:
def labels_to_mask_with_geocube(
    gdf, index, size=256, resolution=1, out_path="./datasets/masks"
):
    from geocube.api.core import make_geocube
    import os

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    # Using GeoCube to rasterize the Vector
    mask_geocube = make_geocube(
        vector_data=gdf,
        measurements=["eunis_niv1_label"],
        resolution=(-resolution, resolution),
        fill=0,
    )
    reduced_data = mask_geocube.isel(x=slice(256), y=slice(256))
    new_da = reduced_data["eunis_niv1_label"].astype("int8")
    new_da
    # Save raster census raster
    new_da.rio.to_raster(f"{out_path}/mask_{index}.tif")
    return f"{out_path}/mask_{index}.tif"


# TODO: refact this function to allow multithread processing
def create_4_band_training_raster(gdf, index, out_path="./datasets/images"):
    import rasterio
    from rasterio.plot import reshape_as_raster, reshape_as_image
    import os

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    (minx, miny, maxx, maxy) = gdf.total_bounds
    ortho_url = get_tile_url(
        base_url="https://wxs.ign.fr/orthohisto/geoportail/r/wms",
        layer_name="ORTHOIMAGERY.ORTHOPHOTOS.ORTHO-EXPRESS.2021",
        bbox=f"{minx},{miny},{maxx},{maxy}",
    )
    irc_url = get_tile_url(
        base_url="https://wxs.ign.fr/orthohisto/geoportail/r/wms",
        layer_name="ORTHOIMAGERY.ORTHOPHOTOS.IRC-EXPRESS.2021",
        bbox=f"{minx},{miny},{maxx},{maxy}",
    )

    if not os.path.exists("./datasets/images"):
        os.makedirs("./datasets/images")

    (ortho_path, ortho_time, d_ortho_url) = download_image_from_url(
        url=ortho_url, index=index, out_path=out_path
    )
    (irc_path, irc_time, d_irc_url) = download_image_from_url(
        url=irc_url, index=index, out_path=out_path
    )
    train_dir = f"{out_path}/rgbn"
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    train_path = f"{train_dir}/image_{index}.tif"
    # Open the RGB image
    with rasterio.open(ortho_path) as rgb_src:
        # Read RGB data and metadata
        rgb_data = rgb_src.read()
        metadata = rgb_src.meta.copy()

    # Open the nir image
    with rasterio.open(irc_path) as infrared_src:
        # Read infrared data
        infrared_data = infrared_src.read(1)

    # Combine RGB and infrared data into a four-band array
    four_band_data = [rgb_data[0], rgb_data[1], rgb_data[2], infrared_data]
    # Update metadata for the new four-band raster
    metadata.update(count=4)

    # Create a new raster file and write the four-band data
    with rasterio.open(train_path, "w", **metadata) as dst:
        for i, band in enumerate(four_band_data, start=1):
            dst.write(band, i)

    return (ortho_path, irc_path, train_path)

# Data preparation


### Remote source files links


In [55]:
# Liens data.montpellier3m.fr
poles_zonage_3m_url = "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_PolesZonage_0.zip"

# cadastre archive contains multiple shapefiles (hydro, bati...)
cadastre_3m_url = (
    "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_Cadastre.zip"
)
contour_3m_url = (
    "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_Contours.zip"
)
occsol_3m_url = (
    "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_OccupSol.zip"
)
vegetation_fine_3m = (
    "https://data.montpellier3m.fr/sites/default/files/ressources/MMM_MMM_VegFine.zip"
)


def download_extract_delete_remote_archives(archives_urls, out_folder="./data/3m"):
    import os
    import requests
    import zipfile

    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

    # Download the archive ; to do serialize this
    for index, url in enumerate(archives_urls):
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Get the filename from the response headers
            content_disposition = response.headers.get("Content-Disposition")
            if content_disposition:
                filename = content_disposition.split("filename=")[1].strip('"')
            else:
                # If no filename is provided in the headers, the endtrail of url
                filename = url.split("/")[-1]

            # Create the path for the downloaded archive
            archive_path = os.path.join(
                out_folder, filename
            )  # Adjust for different archive types

            # Save the downloaded content to the archive file
            with open(archive_path, "wb") as archive_file:
                archive_file.write(response.content)

            # Extract the archive
            extract_folder = os.path.join(
                out_folder, os.path.splitext(os.path.basename(archive_path))[0]
            )
            with zipfile.ZipFile(archive_path, "r") as zip_ref:
                zip_ref.extractall(extract_folder)

            # Optionally, delete the downloaded archive
            os.remove(archive_path)

            print(
                f"Archive downloaded, extracted, and deleted successfully to: {extract_folder}"
            )
        else:
            print(f"Failed to download archive. Status code: {response.status_code}")

In [ ]:
# Download resource files
download_extract_delete_remote_archives(
    archives_urls=[poles_zonage_3m_url, contour_3m_url, occsol_3m_url],
    out_folder=f"{WORKDIR}/data/test",
)

In [57]:
import geopandas as gpd

# Get administrative bondaries for study area (3Md)
boundaries = gpd.read_file(
    "./data/test/MMM_MMM_PolesZonage_0/MMM_MMM_PolesZonage.shp", crs="EPSG:2154"
)

In [59]:
#### (optionnal) create parquet files for big geospatial data (>100Mb) to speed data reads + transformations
import os

# Landcover (occsol) to parquet file
occsol_gdf = gpd.read_file(
    f"{WORKDIR}/data/test/MMM_MMM_OccupSol/MMM_MMM_OccupSol.shp", crs="EPSG:2154"
)
if not os.path.exists(f"{WORKDIR}/data/parquet"):
    os.makedirs(f"{WORKDIR}/data/parquet")
occsol_gdf.to_parquet(f"{WORKDIR}/data/parquet/clc_3m.parquet")

In [60]:
# occsol dataframe preparation
clc_3m_gdf = gpd.read_parquet(f"{WORKDIR}/data/parquet/clc_3m.parquet")
clc_3m_gdf.to_crs("EPSG:2154", inplace=True)

In [61]:
# Create a grid for SA entire coverage
grid_3m = create_size_based_grid(gdf=clc_3m_gdf)
grid_3m["size"] = grid_3m["geometry"].length / 4

In [ ]:
# Uncomment this, if you want plot 3M grid and landcover layer (be patient...)

# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(1, 2, figsize=(14, 7))
# axs = ax.flat
# clc_3m_gdf.plot(ec="gray", fc="none", figsize=(10, 10), ax=axs[0])
# grid_3m.plot(fc="none", ec="black", ax=axs[1])
# axs[0].axis("off")
# axs[1].axis("off")

In [62]:
# Collect landcover fields of interest
clc_21_niv4_3m_gdf = clc_3m_gdf[
    ["c2021_niv1", "lib21_niv1", "c2021_niv4", "lib21_niv4", "geometry"]
]
clc_21_niv4_3m_gdf.head(25)

,c2021_niv1,lib21_niv1,c2021_niv4,lib21_niv4,geometry
0,7,Espaces naturels non boisés,7820,Marais maritimes,"POLYGON ((742269.535 6248240.552, 742250.300 6..."
1,5,Espaces agricoles,5121,Espaces libres urbains,"POLYGON ((783953.823 6291883.226, 783966.394 6..."
2,5,Espaces agricoles,5121,Espaces libres urbains,"POLYGON ((781205.235 6284279.038, 781260.980 6..."
3,8,Eau,8220,Plans d'eau et lagunes littorales,"POLYGON ((780145.613 6277024.020, 780063.022 6..."
4,5,Espaces agricoles,5121,Espaces libres urbains,"POLYGON ((758879.856 6272901.789, 758892.140 6..."
5,5,Espaces agricoles,5221,Vergers en exploitation,"POLYGON ((785921.828 6282821.166, 785954.456 6..."
6,5,Espaces agricoles,5221,Vergers en exploitation,"POLYGON ((760408.100 6274080.600, 760403.112 6..."
7,6,Espaces boisés,6110,Feuillus mésophiles dominants,"POLYGON ((744068.463 6254143.414, 744011.575 6..."
8,8,Eau,8300,"Mer, océan et estuaires","POLYGON ((757311.864 6255758.885, 757309.379 6..."
9,5,Espaces agricoles,5140,Prairies,"POLYGON ((758260.437 6271489.652, 758257.173 6..."


In [63]:
## Add CLC:EUNIS mapping + Stats
import pandas as pd


eunis_mapping_df = pd.read_excel("./data_ns/CORINE_LAND_COVER_MONTPELLIER.xlsx")

map_df = eunis_mapping_df[["c2021_niv4", "eunis_niv1_label"]]
gdf = clc_21_niv4_3m_gdf.merge(eunis_mapping_df, on="c2021_niv4", suffixes=("", "_map"))
eunis_niv1_3m_gdf = gdf[
    ["geometry", "eunis_niv1_label", "CODE_EUNIS_niv1", "c2021_niv4", "c2021_niv1"]
]
eunis_niv1_3m_gdf["area_ha"] = eunis_niv1_3m_gdf["geometry"].area / 10**4
eunis_niv1_3m_gdf.reset_index()
eunis_niv1_3m_gdf.head()

/Users/gaetan/.pyenv/versions/3.10.12/envs/3m-ai-dataprep-env/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,eunis_niv1_label,CODE_EUNIS_niv1,c2021_niv4,c2021_niv1,area_ha
0,"POLYGON ((742269.535 6248240.552, 742250.300 6...",2,B,7820,7,0.005211
1,"POLYGON ((767352.525 6268848.993, 767361.873 6...",2,B,7820,7,0.113166
2,"POLYGON ((763211.640 6263361.392, 763236.168 6...",2,B,7820,7,1.269219
3,"POLYGON ((766041.394 6268039.741, 766037.300 6...",2,B,7820,7,5.224858
4,"POLYGON ((768466.461 6265323.085, 768466.452 6...",2,B,7820,7,0.041889


In [ ]:
# Uncomment this if you want to export spreadsheet with some metrics

# import os

# if not os.path.exists(f"{WORKDIR}/data/export"):
#     os.makedirs(f"{WORKDIR}/data/export")

# df = eunis_niv1_3m_gdf.groupby(
#     ["eunis_niv1_label", "CODE_EUNIS_niv1", "c2021_niv4", "c2021_niv1"]
# )["eunis_niv1_label"].count()
# df.to_csv(f"{WORKDIR}/data/export/eunis1-clc4_2021_3m.csv")
# df.to_excel(f"{WORKDIR}/data/export/eunis1-clc4_2021_3m.xlsx")

In [ ]:
# Uncomment the following lines if you want EUNIS classes metric by city - Excel export only


# boundaries = gpd.read_file(
#     f"{WORKDIR}/data/test/MMM_MMM_PolesZonage_0/MMM_MMM_PolesZonage.shp", crs="EPSG:2154"
# )


# def plot_area_and_grid(area_gdf, grid_gdf, title="Geoplot 1"):
#     import matplotlib.pyplot as plt

#     fig, ax = plt.subplots(figsize=(14, 7))
#     area_gdf.plot(ec="gray", fc="none", figsize=(10, 10), ax=ax)
#     grid_gdf.plot(fc="none", ec="black", ax=ax)
#     ax.set_title(title)


# with pd.ExcelWriter(f"{WORKDIR}/data/export/eunis1-clc4_2021_3m_classes.xlsx") as local_writer:
#     for row in boundaries.itertuples():
#         clipped = eunis_niv1_3m_gdf.clip(row.geometry)
#         clipped_2154 = clipped.to_crs("EPSG:2154")
#         grid = create_size_based_grid(gdf=clipped_2154)
#         plot_area_and_grid(clipped_2154, grid, row.nom)
#         local_summary_df = clipped_2154.groupby(
#             ["eunis_niv1_label", "CODE_EUNIS_niv1"]
#         )["area_ha"].sum()
#         local_summary_df.to_excel(
#             local_writer, sheet_name=f"N={len(grid)} -> {row.nom}"
#         )

# with pd.ExcelWriter(f"{WORKDIR}/data/export/eunis1-clc4_2021_3m.xlsx") as global_writer:
#     global_summary_df = eunis_niv1_3m_gdf.groupby(
#         ["eunis_niv1_label", "CODE_EUNIS_niv1"]
#     )["area_ha"].sum()
#     global_summary_df.to_excel(global_writer)

In [64]:
boundaries = gpd.read_file(
    "./data/test/MMM_MMM_PolesZonage_0/MMM_MMM_PolesZonage.shp", crs="EPSG:2154"
)
cities = [row.nom for row in boundaries.itertuples()]
print(cities)

['CASTELNAU-LE-LEZ', 'MONTPELLIER', 'PEROLS', 'CASTRIES', 'BAILLARGUES', 'BEAULIEU', 'COURNONSEC', 'JACOU', 'CLAPIERS', 'COURNONTERRAL', 'LAVERUNE', 'FABREGUES', 'RESTINCLIERES', 'GRABELS', 'SAINT-BRES', 'JUVIGNAC', 'VENDARGUES', 'LATTES', 'LE CRES', 'MONTFERRIER-SUR-LEZ', 'MONTAUD', 'SAINT-GENIES-DES-MOURGUES', 'MURVIEL-LES-MONTPELLIER', 'SAINT-DREZERY', 'PIGNAN', 'SAUSSAN', 'PRADES-LE-LEZ', 'SUSSARGUES', "SAINT-GEORGES-D'ORQUES", 'SAINT-JEAN-DE-VEDAS', 'VILLENEUVE-LES-MAGUELONE']


In [65]:
boundaries.head()

,codcomm,nom,POLE,geometry
0,340057,CASTELNAU-LE-LEZ,Vallée du Lez,"POLYGON ((771484.390 6283324.480, 771515.720 6..."
1,340172,MONTPELLIER,Montpellier,"POLYGON ((765145.010 6282066.590, 765146.010 6..."
2,340198,PEROLS,Littoral,"POLYGON ((775110.540 6276463.730, 775120.800 6..."
3,340058,CASTRIES,Cadoule et Bérange,"POLYGON ((775699.370 6290396.570, 775814.740 6..."
4,340022,BAILLARGUES,Cadoule et Bérange,"POLYGON ((779570.840 6284434.860, 779691.590 6..."


## Training data preparation utils


In [67]:
def set_h5_for_chunk(
    h5_filename="./datasets/training.h5",
    images_path="./datasets/images",
    masks_path="./datasets/masks",
):
    from PIL import Image
    from pathlib import Path
    import os
    import numpy as np
    import rasterio
    from rasterio.plot import reshape_as_image

    IMAGES_ORTHO_DIR = f"{images_path}/ortho"
    IMAGES_IRC_DIR = f"{images_path}/irc"
    IMAGES_RGBN_DIR = f"{images_path}/rgbn"
    MASKS_DIR = masks_path

    def save_h5_dataset(filename, images_ortho, images_irc, images_training, masks):
        import h5py
        import os

        # ---- Create h5 file
        with h5py.File(filename, "w") as f:
            f.create_dataset("ortho", data=images_ortho)
            f.create_dataset("irc", data=images_irc)
            f.create_dataset("training", data=images_training)
            f.create_dataset("masks", data=masks)

        # ---- done
        size = os.path.getsize(filename) / (1024 * 1024)
        print(
            f"Dataset : {filename}  shape_masks : {masks.shape} shape_training : {images_training.shape} size : {size} Mo   (saved)"
        )

    image_ortho_names = []
    for file in Path(IMAGES_ORTHO_DIR).glob("*.tif"):
        fileName = Path(file).stem + ".tif"
        image_ortho_names.append(os.path.join(IMAGES_ORTHO_DIR, fileName))
        image_ortho_names.sort()

    image_irc_names = []
    for file in Path(IMAGES_IRC_DIR).glob("*.tif"):
        fileName = Path(file).stem + ".tif"
        image_irc_names.append(os.path.join(IMAGES_IRC_DIR, fileName))
        image_irc_names.sort()

    image_rgbn_names = []
    for file in Path(IMAGES_RGBN_DIR).glob("*.tif"):
        fileName = Path(file).stem + ".tif"
        image_rgbn_names.append(os.path.join(IMAGES_RGBN_DIR, fileName))
        image_rgbn_names.sort()

    mask_names = []
    for file in Path(MASKS_DIR).glob("*.tif"):
        fileName = Path(file).stem + ".tif"
        mask_names.append(os.path.join(MASKS_DIR, fileName))
    mask_names.sort()

    images_ortho = [np.asarray(Image.open(image)) for image in image_ortho_names]
    images_irc = [np.asarray(Image.open(image)) for image in image_irc_names]
    images_rgbn = [np.asarray(Image.open(image)) for image in image_rgbn_names]
    masks = [reshape_as_image(rasterio.open(mask).read()) for mask in mask_names]
    image_ortho_datasset = np.array(images_ortho)
    image_irc_datasset = np.array(images_irc)
    image_rgbn_datasset = np.array(images_rgbn)
    mask_dataset = np.array(masks)
    # H5_FILE = "/content/drive/MyDrive/data/data_Montpellier/training_09.h5"
    save_h5_dataset(
        h5_filename,
        image_ortho_datasset,
        image_irc_datasset,
        image_rgbn_datasset,
        mask_dataset,
    )

In [68]:
import numpy as np

for k, g in grid_3m.groupby(np.arange(len(grid_3m)) // 10):
    print(f"##### {k} ######")
    seq_folder = f"{WORKDIR}/datasets/h5/seq{k}"

    for index, row in g.iterrows():
        clipped = eunis_niv1_3m_gdf.clip(row.geometry)
        clipped["eunis_niv1_label"] = clipped["eunis_niv1_label"].astype("int8")
        mask_path = labels_to_mask_with_geocube(
            clipped, index, out_path=f"{seq_folder}/masks"
        )
        (ortho_path, irc_path, train_path) = create_4_band_training_raster(
            clipped, index, out_path=f"{seq_folder}/images"
        )
        print(mask_path)
        set_h5_for_chunk(
            f"{seq_folder}/training_{k}.h5",
            f"{seq_folder}/images",
            f"{seq_folder}/masks",
        )

##### 0 ######
./3m-ai/grid_256px_11_classes/datasets/h5/seq0/masks/mask_0.tif
Dataset : ./3m-ai/grid_256px_11_classes/datasets/h5/seq0/training.h5  shape_masks : (1, 256, 256, 1) shape_training : (1, 256, 256, 4) size : 0.31640625 Mo   (saved)
./3m-ai/grid_256px_11_classes/datasets/h5/seq0/masks/mask_1.tif
Dataset : ./3m-ai/grid_256px_11_classes/datasets/h5/seq0/training.h5  shape_masks : (2, 256, 256, 1) shape_training : (2, 256, 256, 4) size : 0.62890625 Mo   (saved)
./3m-ai/grid_256px_11_classes/datasets/h5/seq0/masks/mask_2.tif
Dataset : ./3m-ai/grid_256px_11_classes/datasets/h5/seq0/training.h5  shape_masks : (3, 256, 256, 1) shape_training : (3, 256, 256, 4) size : 0.94140625 Mo   (saved)
./3m-ai/grid_256px_11_classes/datasets/h5/seq0/masks/mask_3.tif
Dataset : ./3m-ai/grid_256px_11_classes/datasets/h5/seq0/training.h5  shape_masks : (4, 256, 256, 1) shape_training : (4, 256, 256, 4) size : 1.25390625 Mo   (saved)
./3m-ai/grid_256px_11_classes/datasets/h5/seq0/masks/mask_4.tif
D

KeyboardInterrupt: 